In [61]:
import pandas as pd

from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot

In [41]:
file_path = 'USD_BRL Dados Históricos.csv'

# above .data file is comma delimited
dolar_df = pd.read_csv(file_path, sep=",")
dolar_df = dolar_df[::-1].reset_index()
dolar_df = dolar_df.drop(columns=['index'])
dolar_df

,Date,Last,Opening,Max,Min,Var%
0,18.12.1992,"0,0052","0,0052","0,0052","0,0052","-1,89%"
1,21.12.1992,"0,0052","0,0052","0,0052","0,0052","0,00%"
2,22.12.1992,"0,0052","0,0052","0,0052","0,0052","0,00%"
3,23.12.1992,"0,0052","0,0052","0,0052","0,0052","0,00%"
4,24.12.1992,"0,0052","0,0052","0,0052","0,0052","0,00%"
...,...,...,...,...,...,...
7296,29.12.2020,"5,2099","5,2460","5,2473","5,1645","-0,69%"
7297,30.12.2020,"5,1937","5,2051","5,2361","5,1519","-0,31%"
7298,31.12.2020,"5,1935","5,1940","5,1940","5,1940","-0,00%"
7299,01.01.2021,"5,1935","5,1934","5,1935","5,1934","0,00%"


In [49]:
past_features_qt = 3
new_features = [[] for i in range(past_features_qt)]
df_length = len(dolar_df['Last'])
df_step_range = range(past_features_qt, df_length)
for i in df_step_range:
    for j in reversed(range(0, past_features_qt)):
        new_features[j].append(dolar_df['Last'][(i-past_features_qt)+j])
        
features = pd.DataFrame() # dolar_df[['Date', 'Opening', 'Max', 'Min', 'Var%']][past_features_qt:]

for i in range(0, past_features_qt):
    features['t-{}'.format(i+1)] = new_features[past_features_qt-i-1]
    
labels = dolar_df[['Last']][past_features_qt:]

In [53]:
# Aplly Normalization to values of the features
def normalize_features(x):
    return (scale*((x-min_value)/(max_value-min_value)))+new_min

def to_float(x):
    return float(x.replace(',', '.'))
   
for column in features:
    features[column] = features[column].apply(to_float)
    
    new_max = 0.5
    new_min = -0.5
    max_value = features[column].max()
    min_value = features[column].min()
    scale = new_max - new_min
    features[column] = features[column].apply(normalize_features)

features

,t-1,t-2,t-3
0,-0.500000,-0.500000,-0.500000
1,-0.500000,-0.500000,-0.500000
2,-0.500000,-0.500000,-0.500000
3,-0.500000,-0.500000,-0.500000
4,-0.499983,-0.500000,-0.500000
...,...,...,...
7293,0.391037,0.386004,0.385987
7294,0.384882,0.391037,0.386004
7295,0.382128,0.384882,0.391037
7296,0.382094,0.382128,0.384882


In [59]:
test_percent = 10
train_samples_qt = int((dolar_df.shape[0]-past_features_qt)*((100-test_percent)/100))

x_train = features[:][:train_samples_qt]
y_train = labels[:][:train_samples_qt]
x_test = features[:][train_samples_qt:]
y_test = labels[:][train_samples_qt:]

# input_shape = ?

6568

In [ ]:
model = keras.Sequential(
    [
        layers.Dense(64, activation="relu", input_shape=input_shape),
        layers.Dense(64, activation="relu"),
        layers.Dense(1, activation="linear")
    ]
)

model.summary()

In [ ]:
batch_size = 125
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

In [ ]:
loss_values = history.history['loss']
accuracy_values = history.history['accuracy']
epochs = range(1, len(loss_values)+1)

pyplot.plot(epochs, loss_values, label='Training Loss')
pyplot.xlabel('Epochs')
pyplot.ylabel('Loss')
pyplot.legend()

pyplot.show()

pyplot.plot(epochs, accuracy_values, label='Training Accuracy')
pyplot.xlabel('Epochs')
pyplot.ylabel('Accuracy')
pyplot.legend()

pyplot.show()

score = model.evaluate(x_test, y_test, verbose=0)
print('Test\nLoss: {}\nAccuracy: {}'.format(score[0], score[1]))

predictions = model.predict(x_test[:9])
print(predictions)